<a href="https://colab.research.google.com/github/juliocesaria/emotion-detection-nlp/blob/main/02_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Preprocessing – GoEmotions

This notebook handles text preprocessing for the GoEmotions dataset.
Steps include:
- Text cleaning
- Lowercasing
- Removing punctuation
- Preparing data for modeling


In [2]:
print("Preprocessing notebook initialized")


Preprocessing notebook initialized


In [3]:
!pip -q install datasets pandas

import re
import pandas as pd
from datasets import load_dataset


In [4]:
dataset = load_dataset("go_emotions")
train_df = pd.DataFrame(dataset["train"])
val_df   = pd.DataFrame(dataset["validation"])
test_df  = pd.DataFrame(dataset["test"])

print(train_df.shape, val_df.shape, test_df.shape)
train_df.head()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

(43410, 3) (5426, 3) (5427, 3)


,text,labels,id
0,My favourite food is anything I didn't have to...,[27],eebbqej
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
3,To make her feel threatened,[14],ed7ypvh
4,Dirty Southern Wankers,[3],ed0bdzj


In [5]:
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"http\S+|www\.\S+", "", text)     # links
    text = re.sub(r"@\w+", "", text)                # mentions
    text = re.sub(r"#\w+", "", text)                # hashtags
    text = re.sub(r"[^a-z\s']", " ", text)          # keep letters + apostrophe
    text = re.sub(r"\s+", " ", text).strip()        # extra spaces
    return text


In [6]:
for df in [train_df, val_df, test_df]:
    df["text_clean"] = df["text"].apply(clean_text)
    df["len_clean"] = df["text_clean"].apply(len)

train_df[["text", "text_clean", "len_clean", "labels"]].head()


,text,text_clean,len_clean,labels
0,My favourite food is anything I didn't have to...,my favourite food is anything i didn't have to...,58,[27]
1,"Now if he does off himself, everyone will thin...",now if he does off himself everyone will think...,111,[27]
2,WHY THE FUCK IS BAYLESS ISOING,why the fuck is bayless isoing,30,[2]
3,To make her feel threatened,to make her feel threatened,27,[14]
4,Dirty Southern Wankers,dirty southern wankers,22,[3]


In [7]:
before = len(train_df)
train_df = train_df[train_df["text_clean"].str.len() > 0].reset_index(drop=True)
after = len(train_df)

print("Removed empty rows:", before - after)


Removed empty rows: 8
